In [1]:
import pandas as pd
import numpy as np
import time
import torch
import csv
from util_v3 import *
from data_v3 import load_data, get_train_datasets
from model_v3 import *
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# from mscn.data import load_data

import torch
import torch.nn as nn
import torch.nn.functional as F
# from mscn.data import get_train_datasets, load_data, make_dataset

In [2]:
num_classes = 10

## load and encode train data by calling the function

In [3]:
# loading the datasets with num of templates=10
dataset_name = 'tpcds_10'

dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)

Loaded queries
Number of training samples: 9756
Number of validation samples: 1084
Created TensorDataset for training data
Created TensorDataset for validation data


In [4]:
num_epochs = 10
batch_size = 1
hid_units = 256
cuda = False

num_materialized_samples = 0

In [5]:
table2vec, column2vec, op2vec, join2vec = dicts

num_materialized_samples = 0

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

model.train()

SetConv(
  (sample_mlp1): Linear(in_features=26, out_features=256, bias=True)
  (sample_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (predicate_mlp1): Linear(in_features=29, out_features=256, bias=True)
  (predicate_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (join_mlp1): Linear(in_features=56, out_features=256, bias=True)
  (join_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (out_mlp1): Linear(in_features=768, out_features=256, bias=True)
  (out_mlp2): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    accuracy_hist_train = 0

    for batch_idx, data_batch in enumerate(train_data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        optimizer.zero_grad()
        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        is_correct = (torch.argmax(outputs, dim=1) == targets).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_data_loader)

    print(f'Epoch {epoch} Accuracy ' f'{accuracy_hist_train: .2f}')

Epoch 0 Accuracy  0.86
Epoch 1 Accuracy  0.89
Epoch 2 Accuracy  0.51
Epoch 3 Accuracy  0.41
Epoch 4 Accuracy  0.32
Epoch 5 Accuracy  0.11
Epoch 6 Accuracy  0.11
Epoch 7 Accuracy  0.11
Epoch 8 Accuracy  0.11
Epoch 9 Accuracy  0.11


# Training Accuracy

In [7]:
with torch.no_grad():
    correct = 0
    total = 0

    for batch in train_data_loader:
        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)

        _, predicted = torch.max(outputs.data, 1)

        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.1111


# Validation Accuracy

In [9]:
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

model.eval()


with torch.no_grad():
    correct = 0
    total = 0

    for batch in test_data_loader:
        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)

        _, predicted = torch.max(outputs.data, 1)

        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.0000
